# Классификация на Георгия и не Георгия

## Подготовка данных для дальнейшего построение модели

In [1]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from io import BytesIO
import requests
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_g = pd.read_csv("/kaggle/input/george/georges.csv", header=None, names=["image_url"])
df_ng = pd.read_csv("/kaggle/input/non-george/non_georges.csv", header=None, names=["image_url"])
df_g["label"]  = 1
df_ng["label"] = 0
df = pd.concat([df_g, df_ng], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

In [49]:
df_g.shape, df_ng.shape

((2681, 2), (3366, 2))

Дисбаланса классов нет

In [3]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

In [50]:
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as T

def compute_mean_std(dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    total_pixels = 0

    for imgs, _ in loader:
        imgs = imgs.view(imgs.size(0), imgs.size(1), -1)
        total_pixels += imgs.size(0) * imgs.size(2)
        mean += imgs.sum(dim=[0, 2])
        std  += (imgs ** 2).sum(dim=[0, 2])

    mean /= total_pixels
    std = (std / total_pixels - mean ** 2).sqrt()
    return mean.tolist(), std.tolist()


In [52]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

sum_rgb  = torch.zeros(3)
sum2_rgb = torch.zeros(3)
n_pixels = 0

for url in tqdm(df_train["image_url"]):
    try:
        response = requests.get(url, timeout=5)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        tensor = transform(img)  # [3, 224, 224]
        n_pixels += tensor.shape[1] * tensor.shape[2]

        sum_rgb  += tensor.sum(dim=[1, 2])
        sum2_rgb += (tensor ** 2).sum(dim=[1, 2])
    except Exception as e:
        print(f"Ошибка с {url}: {e}")
        continue

mean = sum_rgb / n_pixels
std  = (sum2_rgb / n_pixels - mean**2).sqrt()

print("Mean:", mean.tolist())
print("Std: ", std.tolist())


100%|██████████| 4838/4838 [05:42<00:00, 14.11it/s]


Mean: [0.5541272759437561, 0.4881589412689209, 0.4147355556488037]
Std:  [0.27362990379333496, 0.2739166021347046, 0.27786025404930115]


In [53]:
train_transform = test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

In [54]:
class GeorgeDataset(Dataset):
    def __init__(self, df, transform = None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        url   = self.df.loc[idx, "image_url"]
        label = self.df.loc[idx, "label"]
        try:
            resp = requests.get(url, timeout=5)
            img  = Image.open(BytesIO(resp.content)).convert("RGB")
        except:
            img = Image.new("RGB", (224,224), (0,0,0))
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

In [55]:
train_dataset = GeorgeDataset(df_train, transform=train_transform)
test_dataset  = GeorgeDataset(df_test,  transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Бинарная классификация

Попробуем для начала обучить модель для бинарной классификации. Рассмотрен предобученную модель - **EfficientNet**.

In [12]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-4)

model = model.to(device) 

In [13]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0
    running_corrects = 0
    total = 0

    for imgs, labels in tqdm(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        running_loss += loss.item() * imgs.size(0)
        running_corrects += (preds == labels).sum().item()
        total += imgs.size(0)

    epoch_loss = running_loss / total
    epoch_acc  = running_corrects / total

    model.eval()
    test_loss = 0
    test_corrects = 0
    test_total = 0

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            preds = outputs.argmax(dim=1)
            test_loss += loss.item() * imgs.size(0)
            test_corrects += (preds == labels).sum().item()
            test_total += imgs.size(0)

    epoch_test_loss = test_loss / test_total
    epoch_test_acc  = test_corrects / test_total

    print(
        f"Epoch {epoch}/{num_epochs} — "
        f"Train loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f} | "
        f"Test  loss: {epoch_test_loss:.4f}, acc: {epoch_test_acc:.4f}"
    )


100%|██████████| 152/152 [01:36<00:00,  1.57it/s]


Epoch 1/10 — Train loss: 0.3348, acc: 0.8507 | Test  loss: 0.2005, acc: 0.9231


100%|██████████| 152/152 [01:34<00:00,  1.61it/s]


Epoch 2/10 — Train loss: 0.1377, acc: 0.9508 | Test  loss: 0.1863, acc: 0.9322


100%|██████████| 152/152 [01:32<00:00,  1.65it/s]


...

Epoch 8/10 — Train loss: 0.0278, acc: 0.9899 | Test  loss: 0.2492, acc: 0.9430


100%|██████████| 152/152 [02:59<00:00,  1.18s/it]


Epoch 9/10 — Train loss: 0.0193, acc: 0.9921 | Test  loss: 0.2332, acc: 0.9397


100%|██████████| 152/152 [01:36<00:00,  1.57it/s]
                                               

Epoch 10/10 — Train loss: 0.0191, acc: 0.9921 | Test  loss: 0.2569, acc: 0.9413


Как мы можем заменить модель довольно плохо учиться. Конечно, мы могли бы воспользоваться более лучшими моделями или воспользоваться трюками, которые бы увеличало качество, такими как, к примеру, аугминтация, но стоит также отметить, что учить бинарную задачу для классификации класс и не класс не совсем корректно, поскольку весь негативный класс крайне разнородный и по сути включает в себя «всё, что угодно, кроме одного конкретного класса». В таких условиях бинарная классификация превращается в задачу с плохо определённой границей между классами, и модель начинает запоминать специфические особенности негативных примеров, а не обобщать отличительные черты позитивного класса. Это приводит к переобучению.

Дальнейшем попробуем другие модели для решение нашей задач.

## Получение эмбедингов

Попробуем проебразовать изображение в векторное представление, а уже потом работать с ними. В качестве модели для извлечение эмбедингов возьмом не модель для классификации без последнего слоя, а CLIPE, поскольку CLIP обучен на огромном количестве разнообразных изображений и текстов, и его визуальная часть формирует семантически насыщенные эмбеддинги, хорошо отражающие смысл изображения. Это позволяет эффективно использовать CLIP в задачах, где важна смысловая близость, а не только формальное сходство, как в случае с нашей задачей: обнаружить конкретный класс среди произвольных объектов.


CLIP по своей природе представляет модель, обученной на парных данных: изображения и их текстовые описания. Она оптимизирует контрастивный InfoNCE-лосс, минимизируя расстояние (в косинусной метрике) между эмбеддингами соответствующих изображений и текстов, и одновременно максимизируя расстояние между несоответствующими парами. Благодаря этому CLIP формирует общее латентное пространство, в котором семантически близкие объекты (например, изображения одного класса) оказываются рядом, а объекты разных классов — далеко друг от друга. Такая природа обучения делает эмбеддинги CLIP сразу пригодными для задач, основанных на расстояниях, без дополнительного обучения: k-NN, Mahalanobis, One-Class SVM и другие. Особенно это полезно в сценариях одноклассовой классификации, где нужно отличать позитивные примеры от негативных без явного наличия всех классов в обучающей выборке. Поскольку CLIP уже эффективно разводит разные классы в пространстве, его эмбеддинги позволяют использовать расстояние до положительного эталона (или центра кластера) как меру принадлежности к классу — что делает CLIP мощным инструментом в задачах обнаружения аномалий и одноклассового анализа.

In [57]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import requests
from tqdm import tqdm

In [58]:
class CLIPDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        url   = self.df.loc[idx, "image_url"]
        label = int(self.df.loc[idx, "label"])
        try:
            img = Image.open(BytesIO(requests.get(url, timeout=5).content)).convert("RGB")
        except:
            img = Image.new("RGB", (224, 224), (0, 0, 0))
        return img, label

def collate_fn(batch):
    imgs, labels = zip(*batch)
    return list(imgs), torch.tensor(labels, dtype=torch.long)

df_g  = pd.read_csv("/kaggle/input/george/georges.csv", header=None, names=["image_url"])
df_ng = pd.read_csv("/kaggle/input/non-george/non_georges.csv", header=None, names=["image_url"])
df_g["label"], df_ng["label"] = 1, 0
df = pd.concat([df_g, df_ng], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_train = df.sample(frac=0.8, random_state=42)
df_test  = df.drop(df_train.index)

train_ds = CLIPDataset(df_train)
test_ds  = CLIPDataset(df_test)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:
processor  = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_model.eval()

In [47]:
def extract_clip_embeddings(loader):
    X, y = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(loader):
            inputs = processor(images=imgs, return_tensors="pt").to(device)
            feats  = clip_model.get_image_features(**inputs)
            emb    = F.normalize(feats, p=2, dim=1).cpu().numpy()
            X.append(emb)
            y.extend(labels.numpy())
    return np.vstack(X), np.array(y)

In [48]:
X_train, y_train = extract_clip_embeddings(train_loader)
X_test,  y_test  = extract_clip_embeddings(test_loader)

100%|██████████| 38/38 [01:36<00:00,  2.54s/it]


### Обучение линейных классификаторов

Попробуем обучить простые модели для классификации поверх эмбедингов наших изображений и сравним их работу

In [44]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import normalize

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        class_weight="balanced",
        random_state=42
    ),
    "SVM": SVC(
        kernel="rbf",
        probability=True,
        class_weight="balanced",
        random_state=42
    ),
    "XGBoost": XGBClassifier(
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(128, 64),
        max_iter=300,
        random_state=42
    ),
    "k-NN": KNeighborsClassifier(
        n_neighbors=1,
        metric="cosine"
    )
}

for name, clf in models.items():
    print(f"\n==== {name} ====")
    clf.fit(X_train, y_train)

    y_pred  = clf.predict(X_test)

    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test)[:, 1]
        roc = roc_auc_score(y_test, y_score)
    else:
        y_score = y_pred  # fallback
        roc = roc_auc_score(y_test, y_pred)

    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC:  {roc:.4f}")



==== LogisticRegression ====
Accuracy: 0.9338
ROC-AUC:  0.9777

==== RandomForest ====
Accuracy: 0.9231
ROC-AUC:  0.9796

==== SVM ====
Accuracy: 0.9487
ROC-AUC:  0.9845

==== XGBoost ====
Accuracy: 0.9338
ROC-AUC:  0.9829

==== MLP ====
Accuracy: 0.9512
ROC-AUC:  0.9855

==== k-NN ====
Accuracy: 0.9305
ROC-AUC:  0.9319


In [46]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import normalize

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import normalize

from sklearn.preprocessing import StandardScaler

### попробуем нормализовать векторы
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        class_weight="balanced",
        random_state=42
    ),
    "SVM": SVC(
        kernel="rbf",
        probability=True,
        class_weight="balanced",
        random_state=42
    ),
    "XGBoost": XGBClassifier(
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(128, 64),
        max_iter=300,
        random_state=42
    ),
    "k-NN": KNeighborsClassifier(
        n_neighbors=1,
        metric="cosine"
    )
}

for name, clf in models.items():
    print(f"\n==== {name} ====")
    clf.fit(X_train, y_train)

    y_pred  = clf.predict(X_test)

    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test)[:, 1]
        roc = roc_auc_score(y_test, y_score)
    else:
        y_score = y_pred  # fallback
        roc = roc_auc_score(y_test, y_pred)

    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC:  {roc:.4f}")


==== LogisticRegression ====
Accuracy: 0.9330
ROC-AUC:  0.9762

==== RandomForest ====
Accuracy: 0.9231
ROC-AUC:  0.9796

==== SVM ====
Accuracy: 0.9529
ROC-AUC:  0.9867

==== XGBoost ====
Accuracy: 0.9338
ROC-AUC:  0.9829

==== MLP ====
Accuracy: 0.9529
ROC-AUC:  0.9842

==== k-NN ====
Accuracy: 0.9371
ROC-AUC:  0.9368


### Выводы

В ходе экспериментов были протестированы различные подходы к одноклассовай классификации. Наилучшие результаты по метрике ROC-AUC и accuracy показала полносвязная нейронная сеть (MLP) на эмбидингах, получинных при помощи Clip, что может быть объяснено её способностью эффективно извлекать сложные нелинейные зависимости в эмбеддинговом пространстве. В отличие от одноклассовых подходов, которые оптимизируют геометрическую компактность положительного класса, MLP получает информацию о границе между классами напрямую через бинарные метки, что позволяет ей формировать более точную разделяющую поверхность. Особенно важно отметить, что использование нормализованных эмбеддингов CLIP усиливает эффект семантической разреженности между классами, что дополнительно способствует высокому качеству классификации при использовании моделей, чувствительных к расстоянию (таких как k-NN и MLP). Несмотря на то, что одноклассовые методы концептуально лучше подходят для задач с ограниченным числом классов, в условиях доступности меток и разнообразия классов в тренировочной выборке прямое обучение бинарного классификатора, такого как MLP, может оказаться более эффективным и практически обоснованным решением.